In [2]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [3]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [5]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/japan_geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "adgg7cFtabq2wImcxX4Q9Fum4"
    consumer_secret = "LVuLTaE4FauWr4L6Gwg5HAmq36G9KjZ8QCkTJjiPyuIrWnDZCm"
    access_token = "14324013-Ojr4tv9TGbGIx7yRGWpbozUc4AWhkTdvjIHIBbOME"
    access_token_secret = "B4NR0pssEPvrQIBfum4DNd3q1q4tmUOcJP14xbhVLcVGK"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [129.408463169, 31.0295791692, 145.543137242, 45.5514834662]  

    stream_listener = StreamListener(time_limit=600, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1533653887604822017, 'username': 'Drew__DeLeon', 'created_at': 'Mon Jun 06 03:35:55 +0000 2022', 'lng': 139.745379, 'lat': 35.652789, 'text': '@elizabethgonzzz For some girls, pussy is all they have to offer because they bring no value to the table. Only sim… https://t.co/7xG5I6cru5'}
{'id': 1533654149724016640, 'username': 'naru_is_awake', 'created_at': 'Mon Jun 06 03:36:57 +0000 2022', 'lng': 135.502204, 'lat': 34.7003735, 'text': "Final fantasy ramen 🍜 Doesn't it look cute with them lil 花 (hana ~ flower in japanese) ? I appreciate it if you… https://t.co/mRp0SzDkIx"}
{'id': 1533654181428752387, 'username': '27314photo', 'created_at': 'Mon Jun 06 03:37:05 +0000 2022', 'lng': 139.69262600000002, 'lat': 35.666851, 'text': 'A test shoot for Stefany at Tokyo Rebels from last month (#digital on the left, #film on the right) https://t.co/J8NaM6gWEf'}
{'id': 1533654392754282498, 'username': 'JINandKAI', 'created_at': 'Mon Jun 06 03:37:55 +0000 2022', 'lng': 139.778995, 'lat': 35.6869

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
